Flow for a machine learning project

1. Load configuration
2. Initialize data processor
3. Load and preprocess housing data
4. For each edge connectivity method:
   a. Build graph with specific connectivity
   b. For each model type (Simple/Multi-layer):
      - Initialize model
      - Train model
      - Evaluate on validation set
      - Save results
5. Compare all results
6. Get test result and Generate final report

Imports

In [42]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.reset_peak_memory_stats()

In [43]:
# Imports and Initial Setup
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

# Force reload modules
import importlib
import sys
modules_to_reload = ['config', 'data_processor', 'models', 'trainer', 'graph_builder']
for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

from config import Config
from data_processor import DataProcessor
from models import SimpleGCN, SimpleGAT,MultiLayerGCN
from trainer import Trainer


# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("GNN Real Estate Price Prediction Setup")

GNN Real Estate Price Prediction Setup


In [44]:
import data_processor
import importlib
importlib.reload(data_processor)
from data_processor import DataProcessor

In [45]:
import importlib
import data_processor, models, trainer, config, graph_builder

importlib.reload(data_processor)
importlib.reload(models)
importlib.reload(trainer)
importlib.reload(config)
importlib.reload(graph_builder)

from data_processor import DataProcessor
from models import SimpleGCN, SimpleGAT, MultiLayerGCN
from trainer import Trainer
from config import Config

Feature Engineering Functions

In [46]:
def engineer_features(df):
    """Add engineered features that might improve prediction"""
    df = df.copy()
    
    print("Engineering new features")
    
    # Price per sqft 
    df['price_per_sqft'] = df['price'] / df['sqft_living']
    
    # Total sqft
    df['total_sqft'] = df['sqft_above'] + df['sqft_basement']
    
    # Bathroom to bedroom ratio
    df['bath_bed_ratio'] = df['bathrooms'] / np.maximum(df['bedrooms'], 1)
    
    # Age of house assuming current year is 2015 based on data
    current_year = 2015
    df['house_age'] = current_year - df['yr_built']
    
    # Years since renovation 0 if never renovated
    df['years_since_reno'] = np.where(df['yr_renovated'] == 0, 
                                     df['house_age'], 
                                     current_year - df['yr_renovated'])
    
    # Binary features
    df['has_basement'] = (df['sqft_basement'] > 0).astype(int)
    df['has_been_renovated'] = (df['yr_renovated'] > 0).astype(int)
    df['is_luxury'] = ((df['grade'] >= 10) | (df['waterfront'] == 1) | (df['view'] >= 3)).astype(int)
    
    # Living space efficiency living sqft / lot sqft
    df['space_efficiency'] = df['sqft_living'] / np.maximum(df['sqft_lot'], 1)
    
    print(f"Added 9 engineered features")
    return df

def create_neighborhood_features(df):
    """Create neighborhood-based features using geographic proximity"""
    print("Creating neighborhood features...")
    
    # Create a simple grid-based neighborhood (0.01 degree bins)
    lat_bins = pd.cut(df['lat'], bins=50, labels=False)
    long_bins = pd.cut(df['long'], bins=50, labels=False)
    df['neighborhood_id'] = lat_bins * 50 + long_bins
    
    # Calculate neighborhood statistics
    neighborhood_stats = df.groupby('neighborhood_id')['price'].agg(['mean', 'median', 'std', 'count']).reset_index()
    neighborhood_stats.columns = ['neighborhood_id', 'neighborhood_price_mean', 'neighborhood_price_median', 'neighborhood_price_std', 'neighborhood_count']
    
    # Merge back with original data
    df = df.merge(neighborhood_stats, on='neighborhood_id', how='left')
    
    # Fill NaN values
    df['neighborhood_price_std'] = df['neighborhood_price_std'].fillna(df['price'].std())
    
    print(f"Created neighborhood features for {df['neighborhood_id'].nunique()} neighborhoods")
    return df

print("Feature engineering functions loaded")

Feature engineering functions loaded


Configuration Setup

In [47]:
class ImprovedConfig(Config):
    def __init__(self):
        super().__init__()
        # Add engineered features to embedding features
        self.embedding_features = [
            "bedrooms", "bathrooms", "sqft_living", "floors",
            "grade", "condition", "sqft_above", "sqft_basement",
            # New engineered features
            "total_sqft", "bath_bed_ratio", "house_age", "years_since_reno",
            "has_basement", "has_been_renovated", "is_luxury", "space_efficiency",
            "neighborhood_price_mean", "neighborhood_price_median", "neighborhood_price_std"
        ]
        # Training parameters
        self.default_epochs = 50  # Adjust as needed
        self.default_learning_rate = 0.001

# Initialize configuration
cfg = ImprovedConfig()
processor = DataProcessor(cfg)

print("Configuration Setup Complete")
print(f"   Features to use: {len(cfg.embedding_features)}")
print(f"   Epochs: {cfg.default_epochs}")
print(f"   Learning rate: {cfg.default_learning_rate}")

# Hardware check
print(f"\nHardware Check:")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name()}")
    print(f"   GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print(f"   Using CPU")
print(f"   CPU threads: {torch.get_num_threads()}")

Configuration Setup Complete
   Features to use: 19
   Epochs: 50
   Learning rate: 0.001

Hardware Check:
   CUDA available: True
   GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
   GPU memory: 4.3 GB
   CPU threads: 8


Data Loading and Engineering

In [48]:
print("Loading and Processing Data")

# Load raw data
start_time = time.time()
df = processor.load_raw()
load_time = time.time() - start_time

print(f"Data loaded in {load_time:.2f}s")
print(f"   Shape: {df.shape}")
print(f"   Columns: {len(df.columns)}")

# Show basic statistics
print(f"\nBasic Statistics:")
print(f"   Price range: ${df['price'].min():,.0f} - ${df['price'].max():,.0f}")
print(f"   Mean price: ${df['price'].mean():,.0f}")
print(f"   Median price: ${df['price'].median():,.0f}")

# Feature engineering
feature_start = time.time()
print(f"\nApplying Feature Engineering...")
df = engineer_features(df)
df = create_neighborhood_features(df)
feature_time = time.time() - feature_start

print(f"Feature engineering completed in {feature_time:.2f}s")
print(f"   Enhanced shape: {df.shape}")

# Show sample of new features
new_features = ['price_per_sqft', 'house_age', 'bath_bed_ratio', 'is_luxury', 'neighborhood_price_mean']
print(f"\nSample of new features:")
print(df[new_features].head())

print(f"\nData loading timing:")
print(f"   Raw loading: {load_time:.2f}s")
print(f"   Feature engineering: {feature_time:.2f}s")
print(f"   Total: {load_time + feature_time:.2f}s")

Loading and Processing Data
Loading raw data from: C:/Users/Yuval Rainis/Desktop/School/2025 B/Dlab/Project/kc_final.csv
Loaded 21613 rows and 22 columns
Data loaded in 0.03s
   Shape: (21613, 22)
   Columns: 22

Basic Statistics:
   Price range: $75,000 - $7,700,000
   Mean price: $540,088
   Median price: $450,000

Applying Feature Engineering...
Engineering new features
Added 9 engineered features
Creating neighborhood features...
Created neighborhood features for 880 neighborhoods
Feature engineering completed in 0.01s
   Enhanced shape: (21613, 36)

Sample of new features:
   price_per_sqft  house_age  bath_bed_ratio  is_luxury  \
0             188         60               0          0   
1             209         64               1          0   
2             234         82               0          0   
3             308         50               1          0   
4             304         28               1          0   

   neighborhood_price_mean  
0                  292,615  
1 

Data Preprocessing and Graph Creation

In [ ]:
print(" Data Preprocessing...")

# Create spatial features and preprocess
preprocess_start = time.time()
processor.create_spatial_features(df)
X, y = processor.preprocess(df)
preprocess_time = time.time() - preprocess_start

print(f" Preprocessing completed in {preprocess_time:.2f}s")
print(f"   Features shape: {X.shape}")
print(f"   Labels shape: {y.shape}")


print("\n Creating Train/Val/Test Split...")

# Create indices and split
n_nodes = len(X)
indices = np.arange(n_nodes)
np.random.shuffle(indices)

train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15
train_size = int(train_ratio * n_nodes)
val_size = int(val_ratio * n_nodes)

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

# Build boolean masks
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
test_mask = torch.zeros(n_nodes, dtype=torch.bool)

train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True

print(f"Split created:")
print(f"   Train: {train_mask.sum():,} samples ({train_mask.sum()/n_nodes*100:.1f}%)")
print(f"   Val:   {val_mask.sum():,} samples ({val_mask.sum()/n_nodes*100:.1f}%)")
print(f"   Test:  {test_mask.sum():,} samples ({test_mask.sum()/n_nodes*100:.1f}%)")

print("\n🔧 Building graphs per split (train / val / test)...")
graph_start = time.time()
data_train, data_val, data_test = processor.create_split_graphs(X, y, train_mask, val_mask, test_mask)
graph_time = time.time() - graph_start

print(f" Graphs created in {graph_time:.2f}s")
print(f"   Train Nodes: {data_train.num_nodes:,}, Edges: {data_train.num_edges:,}")
print(f"   Val Nodes:   {data_val.num_nodes:,}, Edges: {data_val.num_edges:,}")
print(f"   Test Nodes:  {data_test.num_nodes:,}, Edges: {data_test.num_edges:,}")

# Wrap for trainer
data_list_train = [data_train]
data_list_val = [data_val]
data_list_test = [data_test]


🔄 Data Preprocessing...
Saved spatial features to: data/spatial_features.csv
Spatial features shape: (21613, 2)
Preprocessing data...
Removed 0 rows with NaN values
Features shape: (21613, 19)
Labels shape: (21613, 1)
Scaled features shape: (21613, 19)
Scaled labels shape: (21613,)
Saved feature statistics to: data/feature_statistics.csv
✅ Preprocessing completed in 0.06s
   Features shape: (21613, 19)
   Labels shape: (21613,)

📊 Creating Train/Val/Test Split...
✅ Split created:
   Train: 15,129 samples (70.0%)
   Val:   3,241 samples (15.0%)
   Test:  3,243 samples (15.0%)

🔧 Building graphs per split (train / val / test)...
🔄 Building separate graphs for Train / Val / Test
  Building train graph with 15129 nodes...
GraphBuilder initialized with concentric radii + similarity filtering
  X shape: (15129, 19), geo shape: (15129, 2), y shape: (15129,)
  Structural features shape: (15129, 5)
  Quality features shape: (15129, 3)
  Building validation graph with 3241 nodes...
GraphBuilder 

Model Training and Comparison

In [ ]:
print("Training Deep Models...")
print("=" * 40)

models_to_try = {
    'GCN': SimpleGCN(input_dim=X.shape[1], hidden_dim=128, dropout=0.4),
    'GAT': SimpleGAT(input_dim=X.shape[1], hidden_dim=128, heads=4, dropout=0.4), 
    'MultiGCN': MultiLayerGCN(input_dim=X.shape[1], hidden_dims=[256, 128, 64, 32, 16], dropout=0.4)
}

best_model = None
best_mape = float('inf')
results = {}

advanced_epochs = 100
advanced_lr = 0.0005

print(f"Training settings: {advanced_epochs} epochs, LR={advanced_lr}")

for model_name, model in models_to_try.items():
    print(f"\n{'='*50}")
    print(f"Training {model_name} Model")
    print(f"{'='*50}")
    
    param_count = sum(p.numel() for p in model.parameters())
    print(f"   Parameters: {param_count:,}")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=advanced_lr, weight_decay=1e-4)
    criterion = torch.nn.MSELoss()

    original_epochs = cfg.default_epochs
    original_lr = cfg.default_learning_rate
    cfg.default_epochs = advanced_epochs
    cfg.default_learning_rate = advanced_lr

    trainer = Trainer(model, optimizer, criterion, cfg)

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        torch.cuda.reset_peak_memory_stats()

    print(f"\nStarting full training...")
    full_training_start = time.time()
    trainer.fit(data_list_train, data_list_val)
    full_training_time = time.time() - full_training_start

    print(f"{model_name} training completed in {full_training_time:.2f}s")

    test_start = time.time()
    test_loss, test_mae, test_rmse = trainer.validate(data_list_test)
    test_time = time.time() - test_start

    model.eval()
    with torch.no_grad():
        data_test_device = data_list_test[0].to(trainer.device)
        predictions = model(data_test_device).cpu().numpy()
        targets = data_test_device.y.cpu().numpy()

    test_pred_orig = processor.label_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
    test_actual_orig = processor.label_scaler.inverse_transform(targets.reshape(-1, 1)).flatten()

    test_mae_orig = np.mean(np.abs(test_pred_orig - test_actual_orig))
    test_rmse_orig = np.sqrt(np.mean((test_pred_orig - test_actual_orig) ** 2))
    relative_errors = np.abs((test_actual_orig - test_pred_orig) / np.maximum(test_actual_orig, 1))
    test_mape = np.mean(relative_errors) * 100
    test_medape = np.median(relative_errors) * 100 

    results[model_name] = {
        'mae': test_mae_orig,
        'rmse': test_rmse_orig,
        'mape': test_mape,
        'medape': test_medape,
        'training_time': full_training_time,
        'test_time': test_time,
        'predictions': test_pred_orig,
        'actual': test_actual_orig,
        'param_count': param_count
    }

    print(f"\n{model_name} Results:")
    print(f"   MAE:    ${test_mae_orig:,.0f}")
    print(f"   RMSE:   ${test_rmse_orig:,.0f}")
    print(f"   MAPE:   {test_mape:.2f}%")
    print(f"   MEDAPE: {test_medape:.2f}%")  
    print(f"   Training time: {full_training_time:.2f}s")
    print(f"   Parameters: {param_count:,}")

    if torch.cuda.is_available():
        memory_used = torch.cuda.max_memory_allocated() / 1e9
        print(f"   GPU memory: {memory_used:.2f} GB")
        torch.cuda.reset_peak_memory_stats()

    if test_mape < best_mape:
        best_mape = test_mape
        best_model = model_name

    cfg.default_epochs = original_epochs
    cfg.default_learning_rate = original_lr

print(f"\nDeep model training completed!")
print(f"Best model: {best_model} with {best_mape:.2f}% MAPE")

Training Deep Models...
SimpleGCN upgraded: 56,097 parameters, 5 conv layers, hidden_dim=128
SimpleGAT upgraded: 429,377 parameters, 4 heads, hidden_dim=128
MultiLayerGCN upgraded: 118,481 parameters, 5 layers, dims=[256, 128, 64, 32, 16]
🔧 Training settings: 100 epochs, LR=0.0005

Training GCN Model
   Parameters: 56,097
Using device: cuda

Starting full training...


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Training Progress:   3%|▎         | 3/100 [00:00<00:14,  6.85it/s, Train Loss=0.0341, Val Loss=0.0106, MAE=0.0905, RMSE=0.1028]

Epoch   1: Train Loss = 0.0725, Val Loss = 0.0147, MAE = 0.1117, RMSE = 0.1213


Training Progress:  13%|█▎        | 13/100 [00:01<00:05, 15.38it/s, Train Loss=0.0092, Val Loss=0.0134, MAE=0.1094, RMSE=0.1157]

...

Epoch  80: Train Loss = 0.0026, Val Loss = 0.0013, MAE = 0.0222, RMSE = 0.0359


Training Progress:  92%|█████████▏| 92/100 [00:07<00:00, 13.17it/s, Train Loss=0.0024, Val Loss=0.0013, MAE=0.0223, RMSE=0.0360]

Epoch  90: Train Loss = 0.0024, Val Loss = 0.0013, MAE = 0.0224, RMSE = 0.0360


Training Progress: 100%|██████████| 100/100 [00:07<00:00, 13.08it/s, Train Loss=0.0023, Val Loss=0.0013, MAE=0.0221, RMSE=0.0360]

Epoch 100: Train Loss = 0.0023, Val Loss = 0.0013, MAE = 0.0221, RMSE = 0.0360

✅ Training completed. Best validation loss: 0.0013
MultiGCN training completed in 7.65s

MultiGCN Results:
   MAE:    $165,532
   RMSE:   $250,254
   MAPE:   36.32%
   MEDAPE: 26.52%
   Training time: 7.65s
   Parameters: 118,481
   GPU memory: 0.80 GB

🏁 Deep model training completed!
🏆 Best model: MultiGCN with 36.32% MAPE
